In [ ]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from sklearn.model_selection import train_test_split

In [ ]:
# 1. 경로 설정
PROCESSED_BASE_PATH = Path('/content/drive/MyDrive/HECTO/Dataset/Celeb_pp')
SAVE_CSV_PATH = '/content/drive/MyDrive/HECTO/Dataset/CSV/celeb_v2_metadata.csv'

data_list = []

print("🔍 폴더 구조 분석 중...")

🔍 폴더 구조 분석 중...


In [ ]:
# 2. 이미지 파일 탐색 (rglob으로 모든 하위 폴더의 jpg 찾기)
image_files = list(PROCESSED_BASE_PATH.rglob('*.jpg'))

for img_path in image_files:
    # 핵심: 전체 경로에서 'PROCESSED_BASE_PATH' 이후의 상대 경로만 추출
    # 예: real/youtube/001/frame_0.jpg
    rel_path = img_path.relative_to(PROCESSED_BASE_PATH)

    # 레이블 결정
    label = 0 if 'real' in str(rel_path).lower() else 1

    # [수정 포인트] Video ID 추출: 이미지가 들어있는 '폴더 경로' 전체를 이름으로 만듦
    # 예: 'real/youtube/001' 폴더에 있다면 -> 'real_youtube_001'이 ID가 됨
    video_id = "_".join(rel_path.parent.parts)

    # 프레임 인덱스 추출 (frame_10.jpg -> 10)
    try:
        frame_idx = int(img_path.stem.split('_')[-1])
    except:
        frame_idx = 0

    data_list.append({
        'dataset_name': 'Celeb-DF-v2',
        'video_id': video_id,
        'frame_idx': frame_idx,
        'frame_path': str(img_path),
        'label': label,
        'bbox': [0, 0, 256, 256],
        'detection_score': 1.0,
        'original_video_width': 1920,
        'original_video_height': 1080
    })

df = pd.DataFrame(data_list)

In [ ]:
# 3. 비디오 단위 Split (8:1:1) - 데이터 누수 방지
print(f"✂️ 총 {df['video_id'].nunique()}개의 비디오를 분할 중...")

unique_videos = df['video_id'].unique()
train_vids, temp_vids = train_test_split(unique_videos, test_size=0.2, random_state=42)
val_vids, test_vids = train_test_split(temp_vids, test_size=0.5, random_state=42)

df.loc[df['video_id'].isin(train_vids), 'split'] = 'train'
df.loc[df['video_id'].isin(val_vids), 'split'] = 'val'
df.loc[df['video_id'].isin(test_vids), 'split'] = 'test'

# 4. 결과 저장
df.to_csv(SAVE_CSV_PATH, index=False)
print(f"✅ CSV 생성 완료: {SAVE_CSV_PATH}")

# 통계 출력
print("\n--- [데이터셋 통계] ---")
print(df.groupby(['split', 'label']).size().unstack(fill_value=0))

✂️ 총 1773개의 비디오를 분할 중...
✅ CSV 생성 완료: /content/drive/MyDrive/HECTO/Dataset/CSV/celeb_v2_metadata.csv

--- [데이터셋 통계] ---
label      0      1
split              
test    1289   1363
train  10421  10582
val     1333   1320
